# Analyze macrophages in segmented cell nuclei

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import anndata
import geopandas as gpd
import scanpy as sc

import pathlib

from tifffile import imread, imwrite
from csbdeep.utils import normalize
#from stardist.models import StarDist2D
#from shapely.geometry import Polygon, Point
from scipy import sparse
from matplotlib.colors import ListedColormap

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import sys
sys.path.append('../../../shared/src')
import nuclei_segmentation_plotting as nsp

import tme_colormaps

In [ ]:
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
mpl.rcParams.update(new_rc_params)

In [ ]:
from importlib import reload
reload(nsp)
#reload(tme_colormaps)

# Import data

In [ ]:
sample_name = '372841_2-1889_7d'

In [ ]:
path_to_data = pathlib.Path('..') / 'data' / 'raw'
path_to_intermediate_data = pathlib.Path('..') / 'data' / 'intermediate'
path_to_intermediate_data.mkdir(parents = True, exist_ok=True)
path_to_results = pathlib.Path('..') / 'results' / sample_name
path_to_results.mkdir(parents = True, exist_ok=True)

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white', figsize = (4,4), dpi_save=300, frameon = False, vector_friendly = False)
sc.settings.figdir = path_to_results
spatial_dot_size = 2.4
spot_size = 100
spatial_transparency = 1

In [ ]:
adata = sc.read(path_to_intermediate_data / 'count_area_filtered_adata.h5ad')

In [ ]:
img_filename = '1730.tif'
img = imread(path_to_data / img_filename)
# Percentile normalization of the image
# Adjust min_percentile and max_percentile as needed
min_percentile = 1
max_percentile = 99#95 too low
img = normalize(img, min_percentile, max_percentile)

In [ ]:
path_to_intermediate_data

In [ ]:
gdf = gpd.read_parquet(path_to_intermediate_data / 'geodataframe.parquet')

In [ ]:
MAC_GENES = ['Csf1r', 'Adgre1', 'Itgam', 'Lyz2'] # 'Cd68', '

In [ ]:
adata.var_names_make_unique()

In [ ]:
sc.tl.score_genes(adata, gene_list = MAC_GENES, score_name='mac_score')

In [ ]:
vmax = adata.obs.mac_score.quantile(0.9)

In [ ]:
nsp.plot_obs_key_and_save_image(title = 'Mac score' , gdf= gdf, img = None, adata= adata, obs_key='mac_score', output_name=path_to_results / "mac_score.tiff", vmin = 0, vmax = vmax)

In [ ]:
adata.obs.mac_score.describe()

In [ ]:
sum(adata.obs.mac_score > 0)

In [ ]:
(adata[:, 'Lyz2'].X > 0).sum()

In [ ]:
adata.n_obs

In [ ]:
sum(adata.obs.mac_score > 0) / adata.n_obs

In [ ]:
sc.pl.violin(adata, keys=['mac_score', 'Lyz2'])

In [ ]:
ad_mac = adata[adata.obs.mac_score > 0]

In [ ]:
ad_mac.uns['log1p']['base'] = None

In [ ]:
ad_mac.shape[0] / adata.shape[0]

In [ ]:
# Identify highly variable genes in the dataset using the Seurat method
sc.pp.highly_variable_genes(ad_mac, flavor="seurat", n_top_genes=2000)
sc.pp.pca(ad_mac)
sc.pp.neighbors(ad_mac)

# Convert polygons to dots for easier plotting

In [ ]:
gdf['centroid'] = gdf.geometry.centroid

In [ ]:
gdf['x'] = gdf.centroid.x
gdf['y'] = gdf.centroid.y

In [ ]:
import seaborn as sns

In [ ]:
ad_mac.obs_names

In [ ]:
coords = gdf[['id', 'x', 'y']]

In [ ]:
coords.set_index(keys = 'id', drop=True, inplace=True)

In [ ]:
coords = coords.loc[ad_mac.obs_names, :]

In [ ]:
ad_mac.obsm['X_spatial'] = coords.to_numpy()

In [ ]:
for gene in ['Spp1', 'Cxcl10', 'H2-Ab1']:
    sc.pl.spatial(ad_mac, color = gene, spot_size=spot_size, size = spatial_dot_size, vmax = 'p99', img = None, cmap = 'Purples')

# Co-expression plots (Spp1, Cxcl9, Cd74)

In [ ]:
df = sc.get.obs_df(ad_mac, keys=['Spp1', 'Cxcl10'])

In [ ]:
#define conditions
conditions = [
    (df['Cxcl10'] > 0) & (df['Spp1'] > 0),
    (df['Cxcl10'] > 0) & (df['Spp1'] == 0),
    (df['Cxcl10'] == 0)& (df['Spp1'] > 0),
    (df['Cxcl10'] == 0)& (df['Spp1'] == 0),
]

#define results
results = ['Cxcl10 & Spp1', 'Cxcl10', 'Spp1', 'none']

#create new column based on conditions in column1 and column2
ad_mac.obs['Cxcl10_Spp1'] = np.select(conditions, results)

In [ ]:
Cxcl10_Spp1_colors= {'Cxcl10':'cyan', 'Spp1':'magenta', 'Cxcl10 & Spp1': 'Purple', 'none': 'white'}

# Fig 2G

In [ ]:
sc.pl.spatial(ad_mac[ad_mac.obs['Cxcl10_Spp1'] == 'Cxcl10'], color = 'Cxcl10_Spp1', img_key=None, spot_size=spot_size, size = spatial_dot_size + 1, alpha = 0.4, palette=Cxcl10_Spp1_colors, save = '_Cxcl10_Spp1_coexpression.png')

In [ ]:
sc.pl.spatial(ad_mac[ad_mac.obs['Cxcl10_Spp1'] == 'Cxcl10 & Spp1'], color = 'Cxcl10_Spp1', img_key=None, spot_size=spot_size, size = spatial_dot_size + 1, alpha = 0.4, palette=Cxcl10_Spp1_colors, save = '_Cxcl10_Spp1_coexpression.png')

In [ ]:
sc.pl.spatial(ad_mac[ad_mac.obs['Cxcl10_Spp1'] == 'Spp1'], color = 'Cxcl10_Spp1', img_key=None, spot_size=spot_size, size = spatial_dot_size + 1, alpha = 0.4, palette=Cxcl10_Spp1_colors, save = '_Cxcl10_Spp1_coexpression.png')

In [ ]:
sc.pl.spatial(ad_mac, color = 'Cxcl10_Spp1', img_key=None, spot_size=spot_size, size = spatial_dot_size + 1, alpha = 0.4, palette=Cxcl10_Spp1_colors, save = '_Cxcl10_Spp1_coexpression.png')

In [ ]:
obs_keys_to_extract = ['leiden_0.5', 'Cxcl10_Spp1']
genes_to_extract = ['Cxcl10', 'Spp1']
coords_to_extract = [('X_spatial', 0),('X_spatial', 1), ('X_umap', 0), ('X_umap', 1)]

In [ ]:
df_export_for_plotting = sc.get.obs_df(ad_mac, keys=obs_keys_to_extract+genes_to_extract, obsm_keys=coords_to_extract)

In [ ]:
df_export_for_plotting

In [ ]:
df_export_for_plotting.to_csv(path_to_results / 'data_export_single_nuclei_{}.csv'.format(sample_name))